# BENCHMARKS

### IMPORT LIBRARIES

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
import re
from tqdm.notebook import tqdm
import torchtext
from anytree import Node
import heapq
from queue import LifoQueue
import queue
import random
import matplotlib.pyplot as plt
import seaborn as sns
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### DATASETS LOADER

In [ ]:
data_folder='LM_datasets'
train_df = pd.read_csv(os.path.join(data_folder, 'train.csv'))
validation_df = pd.read_csv(os.path.join(data_folder, 'validation.csv'))
test_df = pd.read_csv(os.path.join(data_folder, 'test.csv'))

In [3]:
# Prepare test_df for evaluation
test_df_list=[]

# Create a different df for all the records of each unique filename
for filename in test_df['Filename'].unique():
    test_df_list.append(test_df[test_df['Filename']==filename])

print('Number of unique files in test set:', len(test_df_list))

Number of unique files in test set: 119


In [4]:
df_list_train=[]
for filename in train_df['Filename'].unique():
    df_list_train.append(train_df[train_df['Filename']==filename])

### LM model loader

In [65]:
# Custom tokenizer to prepare the data

year_token='YEAR'

def custom_tokenizer(path,MAX_DEPTH):

    # remove leading slash
    path=path.lstrip('/')

    # Split the path into words
    path_words = path.split('/')

    # Trim the path to MAX_DEPTH
    path_words = path_words[:MAX_DEPTH]

    #YEAR token substitution for 4-digit numbers
    for i,tok in enumerate(path_words):
        pattern = r'^\d{4}$'
        if re.match(pattern, tok):
            path_words[i]=year_token

    return path_words

# Function to yield tokens from the DataFrame
def yield_tokens(data_iter,MAX_DEPTH):
    for path in data_iter:
        yield custom_tokenizer(path, MAX_DEPTH)



In [66]:
# Create the vocabulary
def create_vocabulary(MIN_FREQ,MAX_DEPTH):
    vocab = torchtext.vocab.build_vocab_from_iterator(yield_tokens(train_df['Path'],MAX_DEPTH), min_freq=MIN_FREQ)
    vocab.insert_token('<unk>', 0)
    vocab.insert_token('<eos>', 2)
    vocab.insert_token('<sos>', 1)
    vocab.insert_token('<pad>',3)
    vocab.set_default_index(vocab['<unk>'])
    # print(f'len vocab = {len(vocab)}')
    return vocab

In [67]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate,
                tie_weights):

        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        self.tie_weights = tie_weights

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Dropout between embedding and lstm
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                    dropout=dropout_rate, batch_first=True)

        # Dropout between lstm and fc
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)

        if tie_weights:
            assert embedding_dim == hidden_dim, 'cannot tie, check dims'
            self.embedding.weight = self.fc.weight
        self.init_weights()

    def forward(self, src, hidden):
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)
        output = self.dropout(output)
        prediction = self.fc(output)
        return prediction, hidden

    def init_weights(self):
        init_range_emb = 0.1
        init_range_other = 1/math.sqrt(self.hidden_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.embedding_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other)
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hidden_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other)

    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell


    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

In [99]:
#Generate predictions, setting the number of predictions to be generated to prediction_limit

def generate(tokens, MAX_DEPTH, model, custom_tokenizer, vocab, device,prediction_limit, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    # tokens = custom_tokenizer(prompt, MAX_DEPTH)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
    
        src = torch.LongTensor([indices]).to(device)
        prediction, hidden = model(src, hidden)
        # Directly modify the prediction vector to set EOS probability to 0
        eos_index = vocab['<eos>'] # As per your vocab setup
        prediction[:, -1, eos_index] = -float('inf')

        sos_index = vocab['<sos>'] # As per your vocab setup
        prediction[:, -1, sos_index] = -float('inf')
        
        unk_index = vocab['<unk>']
        prediction[:, -1, unk_index] = -float('inf')
        
        probs = torch.softmax(prediction[:, -1], dim=-1)
        
        top_k=prediction_limit
        top_probs, top_indices = torch.topk(probs, top_k)
        # Create a list of tuples (prob, token)
        tokens = vocab.get_itos()
        prob_token_list = [(prob.item(), tokens[i]) for prob, i in zip(top_probs[0], top_indices[0])]
        return prob_token_list


In [69]:
#Load pre-trained models

models_folder='saved_models'

models=[]
for file in os.listdir(models_folder):
    model_state_dict=torch.load(f'saved_models/{file}',map_location=torch.device(device))
    
    # load hyperparameters from the filename
    model_name_without_extension = file.split('.pt')[0]

    # Split the model name by underscore to get the components
    components = model_name_without_extension.split('_')

    # Extract the hyperparameters
    model_prefix = components[0] # This is usually the model type or identifier
    MAX_DEPTH = int(components[1][2:]) # Extract the number after 'MD'
    MIN_FREQ = int(components[2][2:]) # Extract the number after 'MF'
    embedding_size = int(components[3][2:]) # Extract the number after 'es'
    num_layers = int(components[4][2:]) # Extract the number after 'nl'
    dropout_rate = float(components[5][2:]) # Extract the number after 'dr'
    loss = float(components[6][4:])
    vocab= create_vocabulary(MIN_FREQ,MAX_DEPTH)
    print(f'MODEL: {file}, MAX_DEPTH: {MAX_DEPTH}, MIN_FREQ: {MIN_FREQ}, embedding_size: {embedding_size}, num_layers: {num_layers}, dropout_rate: {dropout_rate}, loss: {loss}')
    print(f'vocab_size = {len(vocab)}')
    vocab_size = len(vocab)
    tie_weights = True

    model = LSTM(vocab_size, embedding_size, embedding_size, num_layers, dropout_rate, tie_weights).to(device)
    model.load_state_dict(model_state_dict)
    model.eval()
    models.append((file,model,vocab,MAX_DEPTH))

MODEL: model_MD10_MF3_es128_nl3_dr0.2_loss3.569985.pt, MAX_DEPTH: 10, MIN_FREQ: 3, embedding_size: 128, num_layers: 3, dropout_rate: 0.2, loss: 3.569985
vocab_size = 40717
MODEL: model_MD10_MF5_es128_nl3_dr0.4_loss3.285578.pt, MAX_DEPTH: 10, MIN_FREQ: 5, embedding_size: 128, num_layers: 3, dropout_rate: 0.4, loss: 3.285578
vocab_size = 24507
MODEL: model_MD5_MF3_es256_nl4_dr0.4_loss3.556982.pt, MAX_DEPTH: 5, MIN_FREQ: 3, embedding_size: 256, num_layers: 4, dropout_rate: 0.4, loss: 3.556982
vocab_size = 36724
MODEL: model_MD5_MF5_es512_nl4_dr0.6_loss3.271419.pt, MAX_DEPTH: 5, MIN_FREQ: 5, embedding_size: 512, num_layers: 4, dropout_rate: 0.6, loss: 3.271419
vocab_size = 22170


### Bruteforce algorithm evaluation

#### Tree-builder

In [8]:
# Create the tree
def create_tree_with_occurences(df,root):
    def get_or_create_node(name, parent):
        children = parent.children if parent else []
        for child in children:
            if child.name == name:
                child.count += 1
                return child
        return Node(name, parent=parent, count=1, depth=parent.depth+1)


    paths=df['Path'].tolist()

    for path in paths:
        components = path.split("/")
        current_node = root
        for component in components:
            if component != "":
                current_node = get_or_create_node(component, current_node)
    return root

def create_wordlist_tree(wordlist_file, train_root):
    with open(wordlist_file, 'r') as file:
        wordlist = [line.strip() for line in file]
    root = Node("/")
    
    def add_node(current_node, train_node):
        # Convert train_node.children into a dictionary
        children_dict = {child.name: child for child in train_node.children}

        for word in wordlist:
            if word in children_dict:
                child = Node(word, parent=current_node, count=children_dict[word].count)
                add_node(child, children_dict[word])

    add_node(root, train_root)
    return root

# Create the tree
def create_tree(df):
    def get_or_create_node(name, parent=None):
        children = parent.children if parent else []
        for child in children:
            if child.name == name:
                return child
        return Node(name, parent=parent)


    paths=df['Path'].tolist()

    root = Node("/")
    for path in paths:
        components = path.split("/")
        current_node = root
        for component in components:
            if component != "":
                current_node = get_or_create_node(component, current_node)
    return root

#### Bruteforce depth-breadth algorithm

In [9]:
def simulate_dirbuster(wordlist_file, root, recursive, depth_first, time_1_request, number_of_threads, request_limit):
    with open(wordlist_file, 'r') as file:
        wordlist = [line.strip() for line in file]

    # wordlist.sort()
    total_requests = 0
    successful_responses = 0
    failed_responses = 0
    total_time = 0

    total_requests_list = []
    successful_responses_list = []
    failed_responses_list = []

    def simulate(word, node):
        nonlocal total_requests, successful_responses, failed_responses, total_time
        total_requests += 1
        total_time += time_1_request
        children_map = {c.name: c for c in node.children}
        if word in children_map:
            successful_responses += 1
            if recursive:
                return children_map[word]
        else:
            failed_responses += 1
        return None
    
    q = queue.Queue()

    if depth_first:
        q = LifoQueue()
        
    for word in wordlist:
        q.put((word, root))

    while not q.empty() and total_requests < request_limit:
        word, node = q.get()
        new_node = simulate(word, node)
        if new_node is not None:
            for new_word in wordlist:
                q.put((new_word, new_node))

        if total_requests==1 or (total_requests-1) %20 == 0 :
            # Append the values to the lists
            total_requests_list.append(total_requests)
            successful_responses_list.append(successful_responses)
            failed_responses_list.append(failed_responses)

    if number_of_threads > 1:
        total_time /= number_of_threads
    


    return total_requests_list, successful_responses_list, failed_responses_list, total_time


#### Probability-based directory bruteforce algorithm

In [10]:
def custom_bruteforcer(train_root, root, recursive, depth_first, time_1_request, number_of_threads, request_limit):
    total_requests = 0
    successful_responses = 0
    failed_responses = 0
    total_time = 0

    counter=0

    total_requests_list = []
    successful_responses_list = []
    failed_responses_list = []

    def simulate(word, node):
        
        nonlocal total_requests, successful_responses, failed_responses, total_time
        total_requests += 1
        
        total_time += time_1_request
        children_map = {c.name: c for c in node.children}
        if word in children_map:
            
            successful_responses += 1
            if recursive:
                return children_map[word]
        else:
            failed_responses += 1
        return None

    # Create a max heap with the words in the training tree
    heap = queue.Queue()
    if depth_first:
        heap = LifoQueue()
    children_list=[]
    for node in train_root.children:
        children_list.append((-node.count / sum(c.count for c in train_root.children),node.name,random.randint(0,1000000), node, root))
    children_list.sort(key=lambda x: x[1])
    
    for child in children_list:
        heap.put(child)

    while not heap.empty() and total_requests < request_limit:
        
        _,word,_,node_word,target=heap.get()
        
        new_node = simulate(word, target)
        if new_node is not None:
            children_list=[]
            # Add the children of the newly found directory to the heap
            for child in node_word.children:
                children_list.append((-child.count / sum(c.count for c in node_word.children),child.name,random.randint(0, 1000000), child, new_node))
            
            children_list.sort(key=lambda x: x[1])

            for child in children_list:
                heap.put(child)
            counter+=len(node_word.children)
        if total_requests==1 or (total_requests-1) %20 == 0 :
            # Append the values to the lists
            total_requests_list.append(total_requests)
            successful_responses_list.append(successful_responses)
            failed_responses_list.append(failed_responses)

    if number_of_threads > 1:
        total_time /= number_of_threads
    
    return total_requests_list, successful_responses_list, failed_responses_list, total_time, counter


In [11]:
def probability_bruteforcer(train_root, root, recursive, time_1_request, number_of_threads, request_limit,wordlist_file):
    with open(wordlist_file, 'r') as file:
        wordlist = [line.strip() for line in file]
    found_nodes=[root]

    total_requests = 0
    successful_responses = 0
    failed_responses = 0
    total_time = 0

    counter=0
    
    total_requests_list = []
    successful_responses_list = []
    failed_responses_list = []

    not_redundant_tuples=[]
    def simulate(word, node):
        
        nonlocal total_requests, successful_responses, failed_responses, total_time, found_nodes, not_redundant_tuples
        total_requests += 1
        total_time += time_1_request
        children_map = {c.name: c for c in node.children}
        if word in children_map:
            found_nodes.append(children_map[word])
            not_redundant_tuples.append((node,children_map[word]))
            successful_responses += 1
            if recursive:
                
                return children_map[word]
        else:
            failed_responses += 1
        return None
    # Create a max heap with the words in the training tree
    sum_to_add=sum(c.count for c in train_root.children)
    sum_to_add=max(1,sum_to_add)
    heap = [(-node.count / sum_to_add,node.name,random.randint(0,1000000), node, root) for node in train_root.children]
    heapq.heapify(heap)

    counter=len(heap)

    while len(heap)!=0 and total_requests < request_limit:
        
        _,word,_,node_word,target=heapq.heappop(heap)
        
        new_node = simulate(word, target)
        if new_node is not None:
            # Add the children of the newly found directory to the heap
            for child in node_word.children:
                sum_to_add=sum(c.count for c in node_word.children)
                sum_to_add=max(1,sum_to_add)
                heapq.heappush(heap, (-child.count / sum_to_add,child.name,random.randint(0,1000000), child, new_node))
            counter+=len(node_word.children)
        if total_requests==1 or (total_requests-1) %20 == 0 :
            # Append the values to the lists
            total_requests_list.append(total_requests)
            successful_responses_list.append(successful_responses)
            failed_responses_list.append(failed_responses)
    wordlist_index=0
    node_index=0
    while total_requests < request_limit:
        # print('enter_the_loop')
        if wordlist_index >= len(wordlist):
            wordlist_index=0
            node_index+=1
        if node_index >= len(found_nodes):
            break
        
        children_map = {c.name: c for c in found_nodes[node_index].children}
        if wordlist[wordlist_index] in children_map and (found_nodes[node_index],children_map[wordlist[wordlist_index]]) in not_redundant_tuples:
            wordlist_index+=1
            continue
        
        total_requests += 1
        total_time += time_1_request
        if wordlist[wordlist_index] in children_map:
            successful_responses += 1
            if recursive:
                found_nodes.append(children_map[wordlist[wordlist_index]])
        else:
            failed_responses += 1
        if total_requests==1 or (total_requests-1) %20 == 0 :
            # Append the values to the lists
            total_requests_list.append(total_requests)
            successful_responses_list.append(successful_responses)
            failed_responses_list.append(failed_responses)
        wordlist_index+=1

    if number_of_threads > 1:
        total_time /= number_of_threads
    
    return total_requests_list, successful_responses_list, failed_responses_list, total_time, counter


### Create training tree and wordlist tree to be used for probability based attacks

In [7]:
# Create the training tree
train_root = Node("/", count=1,depth=0)
for df in tqdm(df_list_train, desc="DataFrames to train"):
    train_root = create_tree_with_occurences(df,train_root)

DataFrames to train:   0%|          | 0/419 [00:00<?, ?it/s]

In [8]:
wordlist_trees = []
for wordlist_file in tqdm(os.listdir('chosen_wordlists'), desc="Wordlist Files"):
    wordlist_trees.append(create_wordlist_tree('chosen_wordlists/'+wordlist_file, train_root))

Wordlist Files:   0%|          | 0/4 [00:00<?, ?it/s]

#### LM based algorithm


In [12]:
def lm_bruteforcer(model,vocab,MAX_DEPTH, root, recursive, time_1_request, number_of_threads, request_limit,prediction_limit):
    total_requests = 0
    successful_responses = 0
    failed_responses = 0
    total_time = 0

    total_requests_list = []
    successful_responses_list = []
    failed_responses_list = []

    def simulate(word, node):
        
        nonlocal total_requests, successful_responses, failed_responses, total_time
        total_requests += 1
        
        total_time += time_1_request
        children_map={}
        for c in node.children:
            name=c.name
            pattern = r'^\d{4}$'
            if re.match(pattern, name):
                name=year_token
            children_map[name]=c
            
        if word in children_map:
            successful_responses += 1
            if recursive:
                # return the node with the word associated with it
                return word,children_map[word]
        else:
            failed_responses += 1
        return None, None
    # Create a max heap with the starting prediction of the model
    # random.randint(0,10000)
    predictions=generate(['<sos>'], MAX_DEPTH, model, custom_tokenizer, vocab, device,prediction_limit)
    # print(predictions)
    heap = [(-prob,1,word,['<sos>'], root) for prob,word in predictions]
    heapq.heapify(heap)

    while len(heap)!=0 and total_requests < request_limit:
        
        _,_,word,token_list,target=heapq.heappop(heap)
        
        child_name, new_node = simulate(word, target)
        if new_node is not None:
            new_token_list=token_list+[child_name]
            # Add the children of the newly found directory to the heap
            predictions=generate(new_token_list, MAX_DEPTH, model, custom_tokenizer, vocab, device,prediction_limit)
            
            # print(f'token_list: {new_token_list}, first 10 predictions: {predictions[:10]}')
            for prob,word in predictions:
                heapq.heappush(heap, (-prob,len(new_token_list),word,new_token_list, new_node))
        if total_requests==1 or (total_requests-1) %20 == 0 :
            # Append the values to the lists
            total_requests_list.append(total_requests)
            successful_responses_list.append(successful_responses)
            failed_responses_list.append(failed_responses)

    if number_of_threads > 1:
        total_time /= number_of_threads
    
    return total_requests_list, successful_responses_list, failed_responses_list, total_time, 0

## SIMULATION

In [ ]:
### CREATE TEST WEBSITE TREES
# Prepare test_df for evaluation
test_df_list=[]

# Create a different df for all the records of each unique filename
for filename in test_df['Filename'].unique():
    test_df_list.append(test_df[test_df['Filename']==filename])

print('Number of unique files in test set:', len(test_df_list))

test_roots=[]
# Create test roots:
for df in tqdm(test_df_list, desc="Test trees creation"):
    test_roots.append(create_tree(df))

In [ ]:
# RUN EVERY POSSIBLE SIMULATION!

# Global parameters of the simulation
#
# Specify here the parameters of the simulation:
#
# - recursive: whether the attack methodology should be recursive or not (apply to depth-first and breadth-first)
# - time_1_request: estimated time in ms that takes to send a request and to receive a response (used to estimate the execution time)
# - number_of_threads: number of threads that will be used to simulate the attack (used to estimate the execution time)
# - request_limit: maximum number of requests that will be sent during the simulation (used to estimate the execution time)
#

recursive = True
time_1_request = 150
number_of_threads = 20
request_limit = 100000

categories=list(train_df['Type'].unique()) + ['general']
wordlist_names = [os.path.splitext(os.path.basename(file))[0] for file in os.listdir('chosen_wordlists')]


dfs_toadd=[]

for category in categories:
    print(f'Category: {category}')
    if category=='general':
        print('creating trees')
        df_list_train=[]
        for filename in train_df['Filename'].unique():
            df_list_train.append(train_df[train_df['Filename']==filename])  
        
        train_root = Node("/", count=1,depth=0)
        for df in df_list_train:
            train_root = create_tree_with_occurences(df,train_root)
        
        test_df_list=[]

        # Create a different df for all the records of each unique filename
        for filename in test_df['Filename'].unique():
            test_df_list.append(test_df[test_df['Filename']==filename])
        
        test_roots=[]
        # Create test roots:
        for df in test_df_list:
            test_roots.append(create_tree(df))
        print('done creating trees')
    else:
        print('creating trees')
        local_train_df=train_df[train_df['Type']==category]
        local_test_df=test_df[test_df['Type']==category]

        df_list_train=[]
        for filename in local_train_df['Filename'].unique():
            df_list_train.append(local_train_df[local_train_df['Filename']==filename])  
        
        train_root = Node("/", count=1,depth=0)
        for df in df_list_train:
            train_root = create_tree_with_occurences(df,train_root)
        
        test_df_list=[]

        # Create a different df for all the records of each unique filename
        for filename in local_test_df['Filename'].unique():
            test_df_list.append(local_test_df[local_test_df['Filename']==filename])
        
        test_roots=[]
        # Create test roots:
        for df in test_df_list:
            test_roots.append(create_tree(df))
        print('done creating trees')
    
    wordlist_trees = []
    for wordlist_file in os.listdir('chosen_wordlists'):
        wordlist_trees.append(create_wordlist_tree('chosen_wordlists/'+wordlist_file, train_root))
    for wordlist_name, wordlist_tree in tqdm(zip(wordlist_names, wordlist_trees), desc="Wordlist Trees"):
        for test_root in test_roots:
            # Simulate the breadth approach
            total_requests_list, successful_responses_list, failed_responses_list, total_time = simulate_dirbuster('chosen_wordlists/'+wordlist_name+'.txt', test_root, recursive, False, time_1_request, number_of_threads, request_limit)
            total_request_list=np.array(total_requests_list)
            successful_responses_list=np.array(successful_responses_list)
            failed_responses_list=np.array(failed_responses_list)
            records = np.column_stack((np.repeat(category, len(total_requests_list)), np.repeat('breadth', len(total_requests_list)), np.repeat(wordlist_name, len(total_requests_list)), total_requests_list, successful_responses_list, failed_responses_list))
            dfs_toadd.append(pd.DataFrame(records, columns=['Dataset','Simulation', 'Wordlist', 'Total_requests', 'successful_responses', 'failed_responses']))
            
            # Simulate the depth approach
            total_requests_list, successful_responses_list, failed_responses_list, total_time = simulate_dirbuster('chosen_wordlists/'+wordlist_name+'.txt', test_root, recursive, True, time_1_request, number_of_threads, request_limit)
            total_request_list=np.array(total_requests_list)
            successful_responses_list=np.array(successful_responses_list)
            failed_responses_list=np.array(failed_responses_list)
            records = np.column_stack((np.repeat(category, len(total_requests_list)), np.repeat('depth', len(total_requests_list)),np.repeat(wordlist_name, len(total_requests_list)), total_requests_list, successful_responses_list, failed_responses_list))
            dfs_toadd.append(pd.DataFrame(records, columns=['Dataset','Simulation', 'Wordlist', 'Total_requests', 'successful_responses', 'failed_responses']))
            
            # Simulate probability approach
            total_requests_list, successful_responses_list, failed_responses_list, total_time, counter = probability_bruteforcer(wordlist_tree, test_root, recursive, time_1_request, number_of_threads, request_limit,'chosen_wordlists/'+ wordlist_name+'.txt')
            total_request_list=np.array(total_requests_list)
            successful_responses_list=np.array(successful_responses_list)
            failed_responses_list=np.array(failed_responses_list)
            records = np.column_stack((np.repeat(category, len(total_requests_list)), np.repeat('probability', len(total_requests_list)),np.repeat(wordlist_name, len(total_requests_list)), total_requests_list, successful_responses_list, failed_responses_list))
            dfs_toadd.append(pd.DataFrame(records, columns=['Dataset','Simulation', 'Wordlist', 'Total_requests', 'successful_responses', 'failed_responses']))

    for test_root in tqdm(test_roots, desc="probability"):    
        total_requests_list, successful_responses_list, failed_responses_list, total_time, counter = probability_bruteforcer(train_root, test_root, recursive, time_1_request, number_of_threads, request_limit,'chosen_wordlists/'+ wordlist_name+'.txt')
        total_request_list=np.array(total_requests_list)
        successful_responses_list=np.array(successful_responses_list)
        failed_responses_list=np.array(failed_responses_list)
        records = np.column_stack((np.repeat(category, len(total_requests_list)), np.repeat('probability', len(total_requests_list)) ,np.repeat('train-set', len(total_requests_list)), total_requests_list, successful_responses_list, failed_responses_list))
        dfs_toadd.append(pd.DataFrame(records, columns=['Dataset','Simulation', 'Wordlist', 'Total_requests', 'successful_responses', 'failed_responses']))
            

    for test_root in tqdm(test_roots, desc="LM"):
        model_name, model, vocab, MAX_DEPTH = models[3]
        # Simulate the LM approach
        total_requests_list, successful_responses_list, failed_responses_list, total_time, counter = lm_bruteforcer(model,vocab, MAX_DEPTH, test_root, recursive, time_1_request, number_of_threads, request_limit,prediction_limit=750)
        total_request_list=np.array(total_requests_list)
        successful_responses_list=np.array(successful_responses_list)
        failed_responses_list=np.array(failed_responses_list)
        
        records = np.column_stack((np.repeat(category, len(total_requests_list)), np.repeat('LM', len(total_requests_list)), np.repeat('general', len(total_requests_list)), total_requests_list, successful_responses_list, failed_responses_list))
        dfs_toadd.append(pd.DataFrame(records, columns=['Dataset','Simulation', 'Wordlist', 'Total_requests', 'successful_responses', 'failed_responses']))

In [4]:
# SAVE SIMULATIONS RESULTS IN CSV
benchmark_df=pd.concat(dfs_toadd)
benchmark_df.to_csv('benchmark_results.csv',index=False)

## ANALYSIS ON HOW MANY PREDICTIONS TO CONSIDER IN THE LM-BASED APPROACH

### ATTACK SIMULATIONS USING DIFFERENT VALUES OF PREDICTIONS CONSIDERED


In [ ]:
#global parameters of the simulation
recursive = True
time_1_request = 150
number_of_threads = 20
request_limit = 100000

dfs_toadd=[]

param=[100, 250, 500, 750, 1000, 2000, 5000, 10000]
for prediction_limit in tqdm(param):
    for test_root in tqdm(test_roots, desc="LM simulations"):
        model_name, model, vocab, MAX_DEPTH = models[3]    
        # Simulate the wordlist probability bruteforcer
        total_requests_list, successful_responses_list, failed_responses_list, total_time, counter = lm_bruteforcer(model,vocab, MAX_DEPTH, test_root, recursive, time_1_request, number_of_threads, request_limit,prediction_limit)
        total_request_list=np.array(total_requests_list)
        successful_responses_list=np.array(successful_responses_list)
        failed_responses_list=np.array(failed_responses_list)
    
        records = np.column_stack((np.repeat(prediction_limit, len(total_requests_list)), np.repeat(model_name, len(total_requests_list)), total_requests_list, successful_responses_list, failed_responses_list))
        dfs_toadd.append(pd.DataFrame(records, columns=['Prediction_limit','Simulation', 'Total_requests', 'successful_responses', 'failed_responses']))
param_validation_df = pd.concat(dfs_toadd, ignore_index=True)

In [ ]:
param_validation_df.to_csv('param_validation_results.csv',index=False)

In [ ]:
param_validation_df['Total_requests'] = param_validation_df['Total_requests'].astype(int)
param_validation_df['successful_responses'] = param_validation_df['successful_responses'].astype(int)
param_validation_df['failed_responses'] = param_validation_df['failed_responses'].astype(int)
param_validation_df['Prediction_limit'] = param_validation_df['Prediction_limit'].astype(int)
prediction_limits = param_validation_df['Prediction_limit'].unique()
print(prediction_limits)


# Create 1 figure where we plot the average successful responses for each prediction limit
plt.figure(figsize=(7, 7), dpi=300)
# Define a list of markers
# Define a color palette
palette = sns.color_palette("hsv", len(prediction_limits))

for i, prediction_limit in enumerate(prediction_limits):
    df_simulation = param_validation_df[param_validation_df['Prediction_limit'] == prediction_limit]
    avg_successful_responses = df_simulation.groupby('Total_requests')['successful_responses'].mean().reset_index()
    
    # Plot the average successful responses with a color from the colors list
    plt.plot(avg_successful_responses['Total_requests'], avg_successful_responses['successful_responses'], label=f'topPredicts: {prediction_limit}')

# plt.title('Average Successful Responses for different topPredict values')
plt.xlabel('Total Requests')
plt.ylabel('Successful Responses')
plt.legend()
plt.tight_layout()
plt.savefig('prediction_limit_validation.pdf')
plt.show()
# plt.savefig('prediction_limit_validation.pdf')